<a href="https://colab.research.google.com/github/fromakim/Afterburner/blob/main/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Afterburner Data Collector

고민거리들.
1. OCR을 해야 할까? 일반적인 텍스트에서 유용한 정보가 너무 없고 오히려 이미지에 중요한 정보가 많은 것 같다. 특히, Instagram의 특성상 이미지가 중요해서 더욱 그런 것 같다.
2. Spam 또는 광고를 어떻게 처리해야 할까? ClickBait 문제와 실제 공부 내용이 아닌 광고들이 너무 많은 것 같기도 하다.
3. KoNLPy를 사용하는 것이 옳은 것인지 모르겠다. 너무 신종 언어가 많고 제대로 분리가 안 되는 단어가 태반이다.
4. tomotopy는 MIT License 기반.
5. Crawling을 통하여 만든 Dataset을 인정할 수 있을까?

Benchmark : https://github.com/Keracorn/geulstagram

## 💽 1. Environment Setup

### 1-1. Install Packages

In [ ]:
import time
import json
import re

import pandas as pd

In [ ]:
!pip install konlpy

from konlpy.tag import Okt
okt = Okt()

     |████████████████████████████████| 19.4 MB 1.4 MB/s 
     |████████████████████████████████| 448 kB 45.7 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


### 1-2. Selenium Setup

In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 904 kB 32.2 MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [66.6 kB]
Hit:12 http://ppa.launchpad.net

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
wd = webdriver.Chrome('chromedriver', options=chrome_options)

### 1-3. Global Variables

In [ ]:
NUM_SCROLL = 10

SHORT_WAIT_TIME = 2
MEDIUM_WAIT_TIME = 5
LONG_WAIT_TIME = 10

## 📚 2. Data Collection

### 2-1. Instagram Login

In [ ]:
with open("keys.json", "r") as f:
    keys = json.load(f)

In [ ]:
wd.get('https://www.instagram.com')
time.sleep(MEDIUM_WAIT_TIME)

In [ ]:
username = wd.find_element_by_css_selector('#loginForm > div > div:nth-child(1) > div > label > input')
password = wd.find_element_by_css_selector('#loginForm > div > div:nth-child(2) > div > label > input')
login = wd.find_element_by_css_selector('#loginForm > div > div:nth-child(3) > button')

In [ ]:
username.send_keys(keys['id'])
time.sleep(SHORT_WAIT_TIME)
password.send_keys(keys['pw'])
time.sleep(SHORT_WAIT_TIME)
login.click()
time.sleep(SHORT_WAIT_TIME)

In [ ]:
wd.find_element_by_css_selector('#react-root > section > div > div > div.GA2q- > form > span > button').click()
time.sleep(SHORT_WAIT_TIME)

In [ ]:
wd.find_element_by_css_selector('input').click()
time.sleep(SHORT_WAIT_TIME)

code = input('Keys:')

wd.find_element_by_css_selector('input').send_keys(code)
time.sleep(SHORT_WAIT_TIME)

wd.find_element_by_css_selector('#react-root > section > div > div > div.GA2q- > form > span > button').click()
time.sleep(SHORT_WAIT_TIME)

Keys:635748


### 2-2. Retrieve Post List

In [ ]:
wd.get('https://www.instagram.com/explore/tags/공스타그램/')

In [ ]:
for i in range(NUM_SCROLL):
    wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(LONG_WAIT_TIME)

In [ ]:
posts = wd.find_elements_by_css_selector('#react-root > section > main > article > div:nth-child(3) > div > div > div > a')
posts = pd.DataFrame(list(map(lambda x : x.get_attribute('href'), posts)), columns = ['link'])
print(len(posts))

36


In [ ]:
wd.get('https://www.instagram.com')
time.sleep(LONG_WAIT_TIME)

### 2-3. Retrieve Text

In [ ]:
posts['text'] = ''

In [ ]:
posts

,link,text
0,https://www.instagram.com/p/CTJpZcCHDmd/,💟이미지나 사진만 보고 구매한 랜덤팩에 실망하신적 있나요?🥺\n⠀\n그렇다면 실물을...
1,https://www.instagram.com/p/CTJVOTuBABu/,"Slowly, But Steady\n스터디 그룹 1기를 모집합니다.\n\n1. 조건..."
2,https://www.instagram.com/p/CTIz4TZB0Ov/,도서관 문 열자마자 뛰어오기\n\n.\n\n.\n“\n#공스타그램 #공스타 #공부자...
3,https://www.instagram.com/p/CS7K6KyhdSE/,힘들…어…..\n\n.\n\n.\n“\n#공스타그램 #공스타 #공부자극 #공부스타그...
4,https://www.instagram.com/p/CS6rVmohLJE/,오늘 다 끝낸다\n\n.\n\n.\n“\n#공스타그램 #공스타 #공부자극 #공부스타...
5,https://www.instagram.com/p/CSeH278FleL/,알맹의 소망 떡메모지✨\n.\n여러분들도 개강준비 잘 하고 계신가요?\n알맹의 하늘...
6,https://www.instagram.com/p/CTJu_U3FZnF/,2021.08.28\n.\n.\n.\n.\n.\n\n#공스타그램 #공부 #book ...
7,https://www.instagram.com/p/CTJu9ygBFQz/,cute leaves🌱\n\n—————————\n\nschool starts tom...
8,https://www.instagram.com/p/CTJu7sjF1I-/,2021.08.27\n.\n.\n.\n.\n.\n\n#공스타그램 #공부 #book ...
9,https://www.instagram.com/p/CTJu7l2B5kH/,"""남들 시선 의식하지 말고 너가 행복해 지는 일을 해""\n#square #공부 #공..."


In [ ]:
for index, row in posts.iterrows():
    print('Current Index: (', index, '/', len(posts), ')')

    try:
        wd.get(row['link'])
        time.sleep(MEDIUM_WAIT_TIME)
        
        text = wd.find_element_by_css_selector('#react-root > section > main > div > div.ltEKP > article > div.eo2As > div.EtaWk > ul > div > li > div > div > div.C4VMK > span').get_attribute('innerText')
        posts['text'][index] = text

        time.sleep(MEDIUM_WAIT_TIME)
    except:
        pass


## 🎁 3. Data Preprocessing

In [ ]:
posts['text'].str.replace(' \\n', '', regex = True)

0     💟이미지나 사진만 보고 구매한 랜덤팩에 실망하신적 있나요?🥺⠀그렇다면 실물을 보고 ...
1     Slowly, But Steady스터디 그룹 1기를 모집합니다.1. 조건1) 공스타...
2     도서관 문 열자마자 뛰어오기..“#공스타그램 #공스타 #공부자극 #공부스타그램 #공...
3     힘들…어…....“#공스타그램 #공스타 #공부자극 #공부스타그램 #공스타맞팔 #공스...
4     오늘 다 끝낸다..“#공스타그램 #공스타 #공부자극 #공부스타그램 #공스타맞팔 #공...
5     알맹의 소망 떡메모지✨.여러분들도 개강준비 잘 하고 계신가요?알맹의 하늘색 메모지는...
6     2021.08.28.....#공스타그램 #공부 #book #공부자극 #스터디 #시간...
7     cute leaves🌱—————————school starts tomorrow, I...
8     2021.08.27.....#공스타그램 #공부 #book #공부자극 #스터디 #시간...
9     "남들 시선 의식하지 말고 너가 행복해 지는 일을 해"#square #공부 #공부자...
10    2021.08.26.....#공스타그램 #공부 #book #공부자극 #스터디 #시간...
11    오늘도 나타에 젖은 채로 누워서 주말을 보내려했는데 갑자기 알림이떠서 보니 평일의 ...
12    Best🖤실패 없이 모두 만족하는 바게트백정말수 없이 설명드리는퀄라인으로 합리적인 ...
13    Best🖤실패 없이 모두 만족하는 바게트백정말수 없이 설명드리는퀄라인으로 합리적인 ...
14    Best🖤실패 없이 모두 만족하는 바게트백정말수 없이 설명드리는퀄라인으로 합리적인 ...
15    #공부스타그램📖✏ #공스타그램 Ncs 공부 2일차 추론 문항 다 틀려버리는 참사가 ...
16    2021.08.25.....#공스타그램 #공부 #book #공부자극 #스터디 #시간...
17                                              

In [ ]:
posts['text'].map(okt.morphs)

0     [💟, 이미지, 나, 사진, 만, 보고, 구매, 한, 랜덤, 팩, 에, 실망하신, ...
1     [Slowly, ,, But, Steady, \n, 스터디, 그룹, 1, 기를, 모...
2     [도서관, 문, 열, 자마자, 뛰어오기, \n\n, ., \n\n, ., \n, “...
3     [힘들, …, 어, ….., \n\n, ., \n\n, ., \n, “, \n, #...
4     [오늘, 다, 끝낸다, \n\n, ., \n\n, ., \n, “, \n, #공스타...
5     [알맹, 의, 소망, 떡, 메모, 지, ✨, \n, ., \n, 여러분, 들, 도,...
6     [2021.08, ., 28, \n, ., \n, ., \n, ., \n, ., \...
7     [cute, leaves, 🌱, \n\n, —————————, \n\n, schoo...
8     [2021.08, ., 27, \n, ., \n, ., \n, ., \n, ., \...
9     [", 남, 들, 시선, 의식, 하지, 말고, 너, 가, 행복해, 지는, 일, 을,...
10    [2021.08, ., 26, \n, ., \n, ., \n, ., \n, ., \...
11    [오늘, 도, 나타, 에, 젖은, 채, 로, 누워서, 주말, 을, 보내려, 했는데,...
12    [Best, 🖤, \n, 실패, 없이, 모두, 만족하는, 바게트, 백, \n\n, ...
13    [Best, 🖤, \n, 실패, 없이, 모두, 만족하는, 바게트, 백, \n\n, ...
14    [Best, 🖤, \n, 실패, 없이, 모두, 만족하는, 바게트, 백, \n\n, ...
15    [#공부스타그램, 📖✏, #공스타그램, Ncs, 공부, 2일, 차, 추론, 문항, ...
16    [2021.08, ., 25, \n, ., \n, ., \n, ., \n, ., \...
17                                              

In [ ]:
wd.save_screenshot('current.png')

True